In [2]:
# %cd ./filelists/CUB/
# !ln -s  ../../../CUB_200_2011/images ./images
# %cd ../../

# !ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images

# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data ./filelists/cars/images 
!ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [16]:
import wandb

wandb.init(project="static")

wandb.log({"metric_1": 3}, step=1)
wandb.log({"metric_1": 6}, step=2)
wandb.log({"metric_1": 9}, step=3)

wandb.log({"metric_2": 4}, step=1)

wandb.finish()

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_105423-38lur2nc
wandb: Syncing run vital-violet-13


wandb: WARNING Step must only increase in log calls.  Step 1 < 3; dropping {'metric_2': 4}.
wandb: Waiting for W&B process to finish, PID 31095
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210608_105423-38lur2nc/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20210608_105423-38lur2nc/logs/debug-internal.log
wandb: Run summary:
wandb:     metric_1 9
wandb:        _step 3
wandb:     _runtime 1
wandb:   _timestamp 1623149666
wandb: Run history:
wandb:     metric_1 ▁▅█
wandb:        _step ▁▅█
wandb:     _runtime ▁▁▁
wandb:   _timestamp ▁▁▁
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 1 other file(s)
wandb: 
wandb: Synced vital-violet-13: https://wandb.ai/multi-input/static/runs/38lur2nc


In [13]:
# Plan : 

# Put a run
# List edits to mkae and tests to perform
# Put BG
# Make edits

'/kaggle/working/fsl_ssl'

In [3]:
!python train.py --dataset aircrafts --train_aug --method protonet --committed --stop_epoch 400 --resume --resume_wandb_id vba5sgld

Checkpoint path: ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
Resuming from wandb ID:  vba5sgld
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_171105-vba5sgld
wandb: Resuming run aircrafts protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/vba5sgld
wandb: Run `wandb off` to turn off syncing.

About to start training. Last model and best model will be saved in wandb at every model save. 
 Run save_features.py and test.py after the training completes, with the same arguments
wandb: WARNING Step must only increase in log calls.  Step 32001 < 32820; dropping {'train/loss_proto': 0.17186234891414642}.
wandb: WARNING Step must only i

In [1]:
!python wandb_restore.py --id vba5sgld --path ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/last_model.tar

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_170917-vba5sgld
wandb: Resuming run aircrafts protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/vba5sgld
wandb: Run `wandb off` to turn off syncing.

Checkpoint restored at  ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/last_model.tar
The model's epoch is  319
Please rename it to continue training

wandb: Waiting for W&B process to finish, PID 1642
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210608_170917-vba5sgld/logs/debug.log
wandb: Find internal logs 

In [5]:
# Do again with 200 dimensions
# Do again with 399.tar

!python save_features.py --dataset flowers --train_aug --method protonet --committed

Commit after saving features and testing.
checkpoint_dir: ckpts/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
USE BN: True
outfile is features/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
0/31
10/31
20/31
30/31


In [6]:
!python test.py --dataset flowers --train_aug --method protonet --committed

Testing involves fine-tuning. Commit after testing.
novel_file features/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
600 Test Acc = 89.92% +- 0.51%
